In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('nsaids.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - loss: 1053.7444 - val_loss: 253.0732
Epoch 2/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 291.7875 - val_loss: 234.3577
Epoch 3/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 307.0810 - val_loss: 231.9677
Epoch 4/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 489.5302 - val_loss: 231.6453
Epoch 5/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 447.2698 - val_loss: 241.0101
Epoch 6/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 350.9335 - val_loss: 235.5103
Epoch 7/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 302.8523 - val_loss: 240.3258
Epoch 8/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 316.4536 - val_loss: 242.2652
Epoch 9/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 280.8061 - val_loss: 236.8358
Epoch 10/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 390.3429 - val_loss: 260.6866
Epoch 11/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 332.

271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 184.2450 - val_loss: 202.7260
Epoch 66/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 203.9953 - val_loss: 189.7481
Epoch 67/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 286.8093 - val_loss: 195.0470
Epoch 68/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - loss: 175.6455 - val_loss: 203.3766
Epoch 69/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 206.6574 - val_loss: 186.4765
Epoch 70/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 200.1795 - val_loss: 182.3050
Epoch 71/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 211.4502 - val_loss: 187.0403
Epoch 72/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 170.9704 - val_loss: 218.6368
Epoch 73/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 187.0890 - val_loss: 180.2781
Epoch 74/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step - loss: 237.6098 - val_loss: 179.4718
Epoch 75/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 141.9230 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
                                              Product  Forecasted Quantity
59              Tramadol HCl 50mg (Rounox) 100&#039;s           369.214996
33             Mefenamic Acid (Gopain) Cap 100&#039;s            51.071896
30  Ibuprofen + Paracetamol (Fast Relax) Cap 100&#...            46.935532
25                                Flanax 550mg Tablet            38.985516
1                         Alaxan 525mg Cap 100&#039;s            32.644985
46       Paracetamol (CRISTICOL) 500mg Tab 100&#039;s            31.692961
0                                Advil 200mg Soft Gel            23.